In [1]:
import pandas as pd
import numpy as np
import joblib
from sklearn.preprocessing import LabelEncoder
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Load the feature scaler
scaler = joblib.load("feature_scaler.pkl")

# Define target columns (same as in training)
target_cols = [
    "performance_score",
    "burnout_risk_score"
]

# Load models
models = {}
label_encoders = {}

for target in target_cols:
    model_path = f"model_{target}.pkl"
    models[target] = joblib.load(model_path)
    print(f"Loaded model for {target}")

    # Try to load label encoder if it exists (for classification models)
    try:
        le_path = f"label_encoder_{target}.pkl"
        label_encoders[target] = joblib.load(le_path)
        print(f"Loaded label encoder for {target}")
    except FileNotFoundError:
        print(f"No label encoder for {target} (regression model)")

print("All models loaded successfully!")

Loaded model for performance_score
No label encoder for performance_score (regression model)
Loaded model for burnout_risk_score
Loaded label encoder for burnout_risk_score
All models loaded successfully!


In [3]:
def predict_employee_metrics(employee_data):
    """
    Make predictions for employee metrics.

    Parameters:
    -----------
    employee_data : pd.DataFrame or dict
        Employee features. Should contain all feature columns used during training.
        Can be a single row or multiple rows.

    Returns:
    --------
    pd.DataFrame
        Predictions for all target metrics
    """
    # Convert dict to DataFrame if needed
    if isinstance(employee_data, dict):
        employee_data = pd.DataFrame([employee_data])

    # Make a copy to avoid modifying original data
    data = employee_data.copy()

    # Drop target columns if they exist in the input
    data = data.drop(columns=[col for col in target_cols if col in data.columns], errors='ignore')

    # Scale the features
    X_scaled = scaler.transform(data)

    # Make predictions for each target
    predictions = {}

    for target in target_cols:
        model = models[target]
        pred = model.predict(X_scaled)

        # If classification model, decode the predictions
        if target in label_encoders:
            pred = label_encoders[target].inverse_transform(pred.astype(int))
            predictions[target] = pred

            # Also get probability predictions if available
            if hasattr(model, 'predict_proba'):
                proba = model.predict_proba(X_scaled)
                # Store max probability as confidence
                predictions[f"{target}_confidence"] = proba.max(axis=1)
        else:
            # Regression model
            predictions[target] = pred

    # Create results DataFrame
    results = pd.DataFrame(predictions)

    return results

In [4]:
# Create sample employee data
sample_employee = {
    'meeting_hours_per_week': 9.5,
    'meeting_counts_per_week': 11,
    'messages_sent_per_day': 93,
    'messages_received_per_day': 133,
    'avg_response_latency_min': 9.6,
    'communication_burstiness': 0.37,
    'after_hours_message_ratio': 0.201,
    'communication_balance': 0.69,
    'conversation_length_avg': 12.2,
    'avg_tasks_assigned_per_week': 19,
    'avg_tasks_completed_per_week': 14,
    'task_completion_rate': 0.72,
    'avg_task_age_days': 8.3,
    'overdue_task_ratio': 0.29,
    'task_comment_sentiment_mean': 0.04,
    'logged_hours_per_week': 43.4,
    'variance_in_work_hours': 1.04,
    'late_start_count_per_month': 5,
    'early_exit_count_per_month': 0,
    'absenteeism_rate': 0.044,
    'avg_break_length_minutes': 59.0
}

# Make prediction
predictions = predict_employee_metrics(sample_employee)
print("Predictions for sample employee:")
print(predictions)

Predictions for sample employee:
   performance_score burnout_risk_score  burnout_risk_score_confidence
0           0.635363             medium                       0.987776


In [5]:
# Load test data (using a few rows from the training dataset as example)
test_data = pd.read_csv("dataset/employee_data.csv").head(5)

# Make predictions on batch
batch_predictions = predict_employee_metrics(test_data)

# Combine with original features for comparison
results = pd.concat([
    test_data.reset_index(drop=True),
    batch_predictions
], axis=1)

print("\nBatch Predictions (showing first 5 rows):")
print(results[['performance_score', 'burnout_risk_score'] + list(batch_predictions.columns)].head())


Batch Predictions (showing first 5 rows):
   performance_score  performance_score burnout_risk_score burnout_risk_score  \
0               0.64           0.635363               0.38             medium   
1               0.75           0.747491               0.30                low   
2               0.70           0.709440               0.41             medium   
3               0.72           0.718327               0.53             medium   
4               0.67           0.670477               0.31                low   

   performance_score  performance_score burnout_risk_score burnout_risk_score  \
0               0.64           0.635363               0.38             medium   
1               0.75           0.747491               0.30                low   
2               0.70           0.709440               0.41             medium   
3               0.72           0.718327               0.53             medium   
4               0.67           0.670477               0.31       

In [6]:
def interpret_predictions(predictions_df, employee_id=None):
    """
    Provide human-readable interpretation of predictions.

    Parameters:
    -----------
    predictions_df : pd.DataFrame
        DataFrame with predictions
    employee_id : int, optional
        Index of specific employee to interpret (default: all)
    """
    if employee_id is not None:
        predictions_df = predictions_df.iloc[[employee_id]]

    for idx, row in predictions_df.iterrows():
        print(f"\n{'='*60}")
        print(f"Employee {idx + 1} Analysis:")
        print(f"{'='*60}")

        # Performance Score
        if 'performance_score' in row:
            perf = row['performance_score']
            if isinstance(perf, (int, float)):
                print(f"📊 Performance Score: {perf:.2f}")
                if perf >= 0.7:
                    print("   ✅ High performance - exceeding expectations")
                elif perf >= 0.5:
                    print("   ⚠️  Average performance - meeting expectations")
                else:
                    print("   ⚠️  Below average - may need support")
            else:
                print(f"📊 Performance Score: {perf}")
                if 'performance_score_confidence' in row:
                    print(f"   Confidence: {row['performance_score_confidence']:.2%}")

        # Burnout Risk
        if 'burnout_risk_score' in row:
            burnout = row['burnout_risk_score']
            if isinstance(burnout, (int, float)):
                print(f"🔥 Burnout Risk: {burnout:.2f}")
                if burnout >= 0.6:
                    print("   🚨 HIGH RISK - Immediate intervention recommended")
                elif burnout >= 0.4:
                    print("   ⚠️  MODERATE RISK - Monitor closely and provide support")
                else:
                    print("   ✅ LOW RISK - Employee wellbeing appears healthy")
            else:
                print(f"🔥 Burnout Risk: {burnout}")
                if 'burnout_risk_score_confidence' in row:
                    print(f"   Confidence: {row['burnout_risk_score_confidence']:.2%}")

        print(f"{'='*60}")

# Interpret the sample prediction
interpret_predictions(predictions, employee_id=0)


Employee 1 Analysis:
📊 Performance Score: 0.64
   ⚠️  Average performance - meeting expectations
🔥 Burnout Risk: medium
   Confidence: 98.78%


In [7]:
def get_employee_predictions_api(employee_features):
    """
    API-ready function that returns predictions as a JSON-serializable dictionary.

    Parameters:
    -----------
    employee_features : dict
        Dictionary containing employee feature values

    Returns:
    --------
    dict
        Predictions and interpretations in JSON format
    """
    try:
        # Make predictions
        predictions = predict_employee_metrics(employee_features)

        # Convert to dictionary
        result = {}

        for col in predictions.columns:
            val = predictions[col].iloc[0]
            # Convert numpy types to Python types for JSON serialization
            if isinstance(val, (np.integer, np.floating)):
                result[col] = float(val)
            else:
                result[col] = str(val)

        # Add interpretations
        interpretations = {}

        # Performance interpretation
        if 'performance_score' in result:
            perf = result['performance_score']
            if isinstance(perf, float):
                if perf >= 0.7:
                    interpretations['performance'] = 'high'
                elif perf >= 0.5:
                    interpretations['performance'] = 'average'
                else:
                    interpretations['performance'] = 'low'
            else:
                interpretations['performance'] = perf

        # Burnout interpretation
        if 'burnout_risk_score' in result:
            burnout = result['burnout_risk_score']
            if isinstance(burnout, float):
                if burnout >= 0.6:
                    interpretations['burnout_level'] = 'high'
                elif burnout >= 0.4:
                    interpretations['burnout_level'] = 'moderate'
                else:
                    interpretations['burnout_level'] = 'low'
            else:
                interpretations['burnout_level'] = burnout

        return {
            'predictions': result,
            'interpretations': interpretations,
            'status': 'success'
        }

    except Exception as e:
        return {
            'status': 'error',
            'message': str(e)
        }

# Test the API function
api_result = get_employee_predictions_api(sample_employee)
print("API Response:")
print(api_result)

API Response:
{'predictions': {'performance_score': 0.6353628635406494, 'burnout_risk_score': 'medium', 'burnout_risk_score_confidence': 0.9877759218215942}, 'interpretations': {'performance': 'average', 'burnout_level': 'medium'}, 'status': 'success'}


In [ ]:
# Uncomment to save predictions
# results.to_csv("employee_predictions.csv", index=False)
# print("Predictions saved to employee_predictions.csv")